In [ ]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import torch


In [10]:
from huggingface_hub import login
import os 
from dotenv import load_dotenv


load_dotenv()
LOGIN_TOKEN=os.getenv("login_token")

login(token=LOGIN_TOKEN)



In [19]:
quantization_config= BitsAndBytesConfig(load_in_8bit=True)
use_quantization_config= False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
attention_implementation= 'eager'
model_id='google/gemma-3-1b-it'
tokenizer=AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id,padding_side="left")
llm_model=AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                        torch_dtype=torch.bfloat16,
                                        quantization_config=quantization_config if use_quantization_config else None
                                        ).to(device)


In [20]:
def prompt_gen(query:str):
    base_prompt=f""" 
You are a query expansion specialist for a retrieval system. Your task is to generate multiple search variations of the original query to maximize retrieval of relevant documents.

**Original Query:** "{query}"

Generate exactly 5 expanded queries following these guidelines:

1. **Synonym Variation**: Replace key terms with synonyms and alternative phrasings
2. **Technical Reformulation**: Use domain-specific terminology and technical language
3. **Simplified Version**: Rephrase using common, everyday language
4. **Context Expansion**: Add implicit context or background information that might be relevant
5. **Perspective Shift**: Approach the same information need from a different angle or use case

**Requirements:**
- Keep the core intent and meaning unchanged
- Each variation should be 1-2 sentences maximum
- Focus on terms that would appear in relevant documents
- Avoid redundant variations
- Prioritize searchable keywords over conversational language

**Output Format:**
1. [Synonym variation]
2. [Technical version]
3. [Simplified version]  
4. [Context expanded]
5. [Perspective shifted]

    

"""
    dialogue_template = [
            {"role": "user",
            "content": base_prompt}
        ]
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [21]:
query="why is the protein content in rice is less"
prompt=prompt_gen(query=query)

In [26]:

input_ids = tokenizer(prompt, return_tensors="pt").to(device)

outputs = llm_model.generate(**input_ids,
                             temperature=0.7,
                             do_sample=True, # whether or not to use sampling, see https://huyenchip.com/2024/01/16/sampling.html for more
                             max_new_tokens=1000) # how many new tokens to generate from prompt 

output_text = tokenizer.decode(outputs[0])

In [30]:
text=output_text.replace(prompt, '')

print(f"Query: {query}")
print(f"RAG answer:\n{text}")

Query: why is the protein content in rice is less
RAG answer:
<bos>Okay, here are five expanded queries based on the original query "why is the protein content in rice is less," following your guidelines:

1.  "What factors contribute to reduced protein levels in rice grains?"
2.  "Rice protein yield and nutritional content – what’s the reason for the lower protein?"
3.  "Low protein content in rice: causes and potential explanations."
4.  "Protein analysis of rice: exploring the reasons for diminished levels."
5.  "Impact of processing on rice protein – why is it less?"<end_of_turn>


In [36]:
import re 
matches = re.findall(r"^\s*\d+\.\s+\"(.*?)\"", text, flags=re.MULTILINE)

data=[]
for num,m in enumerate(matches,start=1):
    data.append({f"query{num}":m})


In [ ]:
data

[{'query1': 'What factors contribute to reduced protein levels in rice grains?'},
 {'query2': 'Rice protein yield and nutritional content – what’s the reason for the lower protein?'},
 {'query3': 'Low protein content in rice: causes and potential explanations.'},
 {'query4': 'Protein analysis of rice: exploring the reasons for diminished levels.'},
 {'query5': 'Impact of processing on rice protein – why is it less?'}]